In [1]:
import pandas as pd

df = pd.read_csv("../data/data.csv")
df.head()

,Age,Gender,Education,Income,Debt,Credit_Score,Loan_Amount,Loan_Term,Num_Credit_Cards,Payment_History,Employment_Status,Residence_Type,Marital_Status,Creditworthiness
0,56,Female,Master,149406,34089,581,49200,60,4,Bad,Unemployed,Rented,Single,1
1,69,Female,High School,78896,8626,648,20147,24,7,Good,Employed,Mortgaged,Married,1
2,46,Female,Master,119339,46281,329,41307,12,8,Bad,Unemployed,Owned,Single,1
3,32,Male,High School,131067,29403,816,19019,60,8,Bad,Employed,Owned,Single,1
4,60,Male,PhD,38001,30032,673,16317,36,4,Average,Employed,Rented,Married,0


In [5]:
df["Debt_to_Income"] = df["Debt"] / df["Income"]
df["Credit_Utilization"] = df["Debt"] / df["Loan_Amount"].replace(0, 1)


In [6]:
target_col = "Creditworthiness"
feature_cols = [col for col in df.columns if col != target_col]

X = df[feature_cols]
y = df[target_col]

X.head(), y.head()

(   Age  Gender    Education  Income   Debt  Credit_Score  Loan_Amount  \
 0   56  Female       Master  149406  34089           581        49200   
 1   69  Female  High School   78896   8626           648        20147   
 2   46  Female       Master  119339  46281           329        41307   
 3   32    Male  High School  131067  29403           816        19019   
 4   60    Male          PhD   38001  30032           673        16317   
 
    Loan_Term  Num_Credit_Cards Payment_History Employment_Status  \
 0         60                 4             Bad        Unemployed   
 1         24                 7            Good          Employed   
 2         12                 8             Bad        Unemployed   
 3         60                 8             Bad          Employed   
 4         36                 4         Average          Employed   
 
   Residence_Type Marital_Status  Debt_to_Income  Credit_Utilization  
 0         Rented         Single        0.228164            0.69286

In [7]:
numeric_features = [
    "Age", "Income", "Debt", "Credit_Score",
    "Loan_Amount", "Loan_Term", "Num_Credit_Cards",
    "Debt_to_Income", "Credit_Utilization",
]
categorical_features = ["Gender", "Education", "Payment_History",
                        "Employment_Status", "Residence_Type", "Marital_Status"]

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

X_train.shape, X_test.shape

((9600, 15), (2400, 15))

In [10]:
from lightgbm import LGBMClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

lgbm_clf = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", LGBMClassifier(
            n_estimators=500,
            learning_rate=0.05,
            class_weight="balanced",
            random_state=42,
        )),
    ]
)

lgbm_clf.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 6745, number of negative: 2855
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000300 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1631
[LightGBM] [Info] Number of data points in the train set: 9600, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [11]:
feature_names = (
    lgbm_clf.named_steps["preprocess"]
    .get_feature_names_out()
)


In [12]:
importances = lgbm_clf.named_steps["model"].feature_importances_

In [13]:
fi = pd.DataFrame({
    "feature": feature_names,
    "importance": importances
}).sort_values("importance", ascending=False)

fi.head(20), fi.tail(20)

(                                 feature  importance
 3                      num__Credit_Score        1964
 4                       num__Loan_Amount        1722
 1                            num__Income        1676
 8                num__Credit_Utilization        1561
 2                              num__Debt        1513
 0                               num__Age        1420
 7                    num__Debt_to_Income        1409
 6                  num__Num_Credit_Cards         733
 5                         num__Loan_Term         492
 9                     cat__Gender_Female         211
 22             cat__Residence_Type_Owned         183
 20     cat__Employment_Status_Unemployed         161
 24          cat__Marital_Status_Divorced         154
 26            cat__Marital_Status_Single         154
 23            cat__Residence_Type_Rented         153
 21         cat__Residence_Type_Mortgaged         151
 25           cat__Marital_Status_Married         145
 18       cat__Employment_St

In [14]:
numeric_features_reduced = [
    "Age", "Income", "Debt", "Credit_Score",
    "Loan_Amount", "Debt_to_Income", "Credit_Utilization",
]

categorical_features_reduced = [
    "Payment_History", "Employment_Status", "Residence_Type", "Marital_Status",
]

preprocess_reduced = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features_reduced),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features_reduced),
    ]
)

lgbm_clf_reduced = Pipeline(
    steps=[
        ("preprocess", preprocess_reduced),
        ("model", LGBMClassifier(
            n_estimators=500,
            learning_rate=0.05,
            class_weight="balanced",
            random_state=42,
        )),
    ]
)

lgbm_clf_reduced.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

y_pred = lgbm_clf_reduced.predict(X_test)
y_proba = lgbm_clf_reduced.predict_proba(X_test)[:, 1]

print("Accuracy:", accuracy_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))


[LightGBM] [Info] Number of positive: 6745, number of negative: 2855
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000463 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1603
[LightGBM] [Info] Number of data points in the train set: 9600, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Accuracy: 0.59875
ROC-AUC: 0.49502493761442895
              precision    recall  f1-score   support

           0       0.32      0.30      0.31       714
           1       0.71      0.73      0.72      1686

    accuracy                           0.60      2400
   macro avg       0.51      0.51      0.51      2400
weighted avg       0.59      0.60      0.60      2400



c:\Users\danil\OneDrive\Desktop\Github\ml-credit-scoring\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
c:\Users\danil\OneDrive\Desktop\Github\ml-credit-scoring\.venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
